# Low cost GNSS Data quality

In this section we will check 


The data provided hee corresponds to 



In [ ]:
import matplotlib.pyplot as plt

from roktools import rinex

In [ ]:
df_geodetic = rinex.to_dataframe('../assets/SEYG00SYC_R_20140581500_05H_01S_MO.rnx')


In [ ]:
df_ublox = rinex.to_dataframe('../assets/MTIC00ESP_R_20191221131_05H_01S_MO.rnx')

## Observable types

Once we have loaded the RINEX files into DataFrames, we can perform some
basic checks on the differences between geodetic and affordable GNSS data, for
instance the type of observables generated by one or the other.


In [ ]:
columns = ['constellation', 'channel']

# Use groupby() to group by the two columns and apply unique()
unique_combinations = df_geodetic.groupby(columns).size()

# Print the unique combinations
print(unique_combinations)

In [ ]:
# Use groupby() to group by the two columns and apply unique()
unique_combinations = df_ublox.groupby(columns).size()

# Print the unique combinations
print(unique_combinations)

In these examples, the geodetic receiver tracks various frequencies (GPS L1/L2/L5, Galileo E1/E5a/E5b/E5, ...) whereas the affordable receiver tracks typically two frequencies (GPS L1/L2, Galileo E1/E5b, ...)

Some other strenghts of affordable receivers:

- Availability of SNR and Doppler measurements (not always available in 30s or high rate CORS data)
- High rate up to 0.1s (or even higher) available for affordable measurements



## Observable noise: Detrended LI

A basic quality parameter we can check is the computation of the detrended ionospheric observable
observation. The ionospheric combination removes the geometry, while the detrending process 
removes the ionosphere as well as the biases (ambiguities and hardware biases).
The remaining terms is an approximation of the phase noise (if LI is used) 
or the code noise (if PI is used instead)

In [ ]:
import pandas as pd
from roktools.gnss.types import ConstellationId, TrackingChannel

def compute_cmc(df: pd.DataFrame, constellation: ConstellationId, channel_a: TrackingChannel, channel_b: TrackingChannel) -> pd.DataFrame:
    """
    Compute the Code minus Carrier
    """

    # Create subsets of the DataFrame corresponding to the constellation and each of
    # the channels selected to build the ionospheric combination
    df_a = df[(df['constellation'] == constellation) & (df['channel'] == str(channel_a))]
    df_b = df[(df['constellation'] == constellation) & (df['channel'] == str(channel_b))]

    # Compute the wavelength of the two tracking channels
    wl_a = channel_a.get_wavelength(constellation)
    wl_b = channel_b.get_wavelength(constellation)
    
    # Use merge to join the two tables
    df_out = pd.merge(df_a, df_b, on=['epoch', 'sat'], how='inner', suffixes=('_a', '_b'))
    df_out['li_m'] = df_out['phase_a'] * wl_a - df_out['phase_b'] * wl_b
    df_out['pi_m'] = df_out['range_b'] - df_out['range_a']

    # Define a custom rolling function to calculate the trend
    def do_rolling_window(group, column, new_column):
        group[new_column] = group[column].rolling(window=30).mean()
        return group

    # Apply the custom rolling function within each group
    df_tmp = df_out.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'li_m', 'li_trend_m'))
    b = df_tmp.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'pi_m', 'pi_trend_m'))

    return b

def get_seconds_since_start(epochs:pd.Series) -> list:
    """
    Converts a pandas series of epochs into a list of seconds relative to the first epoch
    """
    return (epochs - epochs[0]).dt.total_seconds().tolist()


In [ ]:

satsys = ConstellationId.GPS
ch_a = TrackingChannel(1, 'C')
ch_b = TrackingChannel(2, 'W')
df_geodetic_li = compute_cmc(df_geodetic, satsys, ch_a, ch_b)

ch_a = TrackingChannel(1, 'C')
ch_b = TrackingChannel(2, 'C')
df_ublox_li = compute_cmc(df_ublox, satsys, ch_a, ch_b)



In [ ]:
a = df_geodetic_li.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'li_m', 'li_trend_m'))

In [ ]:
a.groupby('sat', group_keys=False).apply(lambda group: do_rolling_window(group, 'pi_m', 'pi_trend_m'))

In [ ]:

plt.close()
seconds = get_seconds_since_start(df_geodetic_li['epoch'])
plt.plot(seconds, df_geodetic_li['li_m'] - df_geodetic_li['li_trend_m'], '.')

seconds = get_seconds_since_start(df_ublox_li['epoch'])
plt.plot(seconds, df_ublox_li['li_m'] - df_ublox_li['li_trend_m'], '.')


